[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/rag-chatbot.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/langchain/rag-chatbot.ipynb)

# Building RAG Chatbots with LangChain

In this example, we'll work on building an AI chatbot from start-to-finish. We will be using LangChain, OpenAI, and Pinecone vector DB, to build a chatbot capable of learning from the external world using **R**etrieval **A**ugmented **G**eneration (RAG).

We will be using a dataset sourced from the Llama 2 ArXiv paper and other related papers to help our chatbot answer questions about the latest and greatest in the world of GenAI.

By the end of the example we'll have a functioning chatbot and RAG pipeline that can hold a conversation and provide informative responses based on a knowledge base.

### Before you begin

You'll need to get an [OpenAI API key](https://platform.openai.com/account/api-keys) and [Pinecone API key](https://app.pinecone.io).

### Prerequisites

Before we start building our chatbot, we need to install some Python libraries. Here's a brief overview of what each library does:

- **langchain**: This is a library for GenAI. We'll use it to chain together different language models and components for our chatbot.
- **openai**: This is the official OpenAI Python client. We'll use it to interact with the OpenAI API and generate responses for our chatbot.
- **datasets**: This library provides a vast array of datasets for machine learning. We'll use it to load our knowledge base for the chatbot.
- **pinecone-client**: This is the official Pinecone Python client. We'll use it to interact with the Pinecone API and store our chatbot's knowledge base in a vector database.

You can install these libraries using pip like so:

In [ ]:
# !pip install -qU \
#     langchain==0.0.292 \
#     openai==0.28.0 \
#     datasets==2.10.1 \
#     pinecone-client==2.2.4 \
#     tiktoken==0.5.1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flagembedding 1.1.8 requires transformers==4.34.0, but you have transformers 4.36.2 which is incompatible.
ragas 0.0.22 requires openai>1, but you have openai 0.28.0 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Building a Chatbot (no RAG)

We will be relying heavily on the LangChain library to bring together the different components needed for our chatbot. To begin, we'll create a simple chatbot without any retrieval augmentation. We do this by initializing a `ChatOpenAI` object. For this we do need an [OpenAI API key](https://platform.openai.com/account/api-keys).

In [4]:
import os
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or "YOUR_API_KEY"

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'
)

Chats with OpenAI's `gpt-3.5-turbo` and `gpt-4` chat models are typically structured (in plain text) like this:

```
System: You are a helpful assistant.

User: Hi AI, how are you today?

Assistant: I'm great thank you. How can I help you?

User: I'd like to understand string theory.

Assistant:
```

The final `"Assistant:"` without a response is what would prompt the model to continue the conversation. In the official OpenAI `ChatCompletion` endpoint these would be passed to the model in a format like:

```python
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hi AI, how are you today?"},
    {"role": "assistant", "content": "I'm great thank you. How can I help you?"}
    {"role": "user", "content": "I'd like to understand string theory."}
]
```

In LangChain there is a slightly different format. We use three _message_ objects like so:

In [5]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

The format is very similar, we're just swapped the role of `"user"` for `HumanMessage`, and the role of `"assistant"` for `AIMessage`.

We generate the next response from the AI by passing these messages to the `ChatOpenAI` object.

In [6]:
res = chat(messages)
res

AIMessage(content="String theory is a theoretical framework in physics that aims to explain the fundamental nature of the universe. It suggests that the fundamental building blocks of all matter and energy are tiny vibrating strings, rather than particles. These strings are incredibly small, on the order of the Planck length, which is about 10^-35 meters.\n\nThe theory suggests that the different vibrational patterns of these strings give rise to the various particles and forces we observe in the universe. For example, particles like electrons and quarks are different vibrational modes of the strings. The interactions between these particles are also explained by the vibrations and interactions between the strings.\n\nString theory also proposes a higher-dimensional universe, beyond the three spatial dimensions we are familiar with. It suggests that there may be additional compact dimensions that are too small to be detected directly. These extra dimensions play a crucial role in the m

In response we get another AI message object. We can print it more clearly like so:

In [ ]:
print(res.content)

Sure, I can help you with that. String theory is a theoretical framework in physics that aims to describe the fundamental particles and forces of nature. It suggests that the fundamental building blocks of the universe are not point-like particles, but rather tiny, vibrating strings.

According to string theory, these strings can vibrate in different ways, and the specific vibration modes determine the properties of the particles they represent. For example, the different vibration modes of a string can give rise to particles with different masses and charges.

One of the key ideas in string theory is that it requires more than the three spatial dimensions we are familiar with. In fact, string theory suggests that there are additional spatial dimensions beyond the three we perceive. These extra dimensions are thought to be curled up or compactified at very small scales, making them undetectable in our everyday experience.

String theory also proposes that there are different versions o

Because `res` is just another `AIMessage` object, we can append it to `messages`, add another `HumanMessage`, and generate the next response in the conversation.

In [ ]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do physicists believe it can produce a 'unified theory'?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Physicists believe that string theory has the potential to provide a unified theory because it attempts to explain all the fundamental forces and particles of nature within a single framework. 

In the Standard Model of particle physics, there are four fundamental forces: gravity, electromagnetism, and the strong and weak nuclear forces. These forces are described by different mathematical equations and have different characteristics. 

One of the main motivations behind string theory is to find a way to reconcile the fundamental forces and particles described by the Standard Model with the principles of general relativity, which describes gravity on a larger scale. General relativity and quantum mechanics, which governs the behavior of particles on very small scales, are currently described by separate theories that don't easily fit together.

String theory proposes that all the particles and forces in the universe can be understood as different vibrational modes of tiny strings. By d

### Dealing with Hallucinations

We have our chatbot, but as mentioned — the knowledge of LLMs can be limited. The reason for this is that LLMs learn all they know during training. An LLM essentially compresses the "world" as seen in the training data into the internal parameters of the model. We call this knowledge the _parametric knowledge_ of the model.

By default, LLMs have no access to the external world.

The result of this is very clear when we ask LLMs about more recent information, like about the new (and very popular) Llama 2 LLM.

In [ ]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What is so special about Llama 2?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [ ]:
print(res.content)

I'm sorry, but I'm not familiar with what you're referring to as "Llama 2." Could you please provide more context or clarify your question?


Our chatbot can no longer help us, it doesn't contain the information we need to answer the question. It was very clear from this answer that the LLM doesn't know the informaiton, but sometimes an LLM may respond like it _does_ know the answer — and this can be very hard to detect.

OpenAI have since adjusted the behavior for this particular example as we can see below:

In [ ]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Can you tell me about the LLMChain in LangChain?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [ ]:
print(res.content)

I apologize, but I am not familiar with specific details about the LLMChain in LangChain. It seems to be a specific component or feature within the LangChain system or project. Without more information, it is challenging for me to provide a detailed explanation.

If you can provide more context or specific details about the LLMChain in LangChain, I'll do my best to assist you further.


There is another way of feeding knowledge into LLMs. It is called _source knowledge_ and it refers to any information fed into the LLM via the prompt. We can try that with the LLMChain question. We can take a description of this object from the LangChain documentation.

In [ ]:
llmchain_information = [
    "A LLMChain is the most common type of chain. It consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser. This chain takes multiple input variables, uses the PromptTemplate to format them into a prompt. It then passes that to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format.",
    "Chains is an incredibly generic concept which returns to a sequence of modular components (or other chains) combined in a particular way to accomplish a common use case.",
    "LangChain is a framework for developing applications powered by language models. We believe that the most powerful and differentiated applications will not only call out to a language model via an api, but will also: (1) Be data-aware: connect a language model to other sources of data, (2) Be agentic: Allow a language model to interact with its environment. As such, the LangChain framework is designed with the objective in mind to enable those types of applications."
]

source_knowledge = "\n".join(llmchain_information)

We can feed this additional knowledge into our prompt with some instructions telling the LLM how we'd like it to use this information alongside our original query.

In [ ]:
query = "Can you tell me about the LLMChain in LangChain?"

augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""

Now we feed this into our chatbot as we were before.

In [ ]:
# create a new user prompt
prompt = HumanMessage(
    content=augmented_prompt
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [ ]:
print(res.content)

In the context of LangChain, an LLMChain is a type of chain within the LangChain framework. A chain in LangChain refers to a sequence of modular components or other chains that are combined in a specific way to achieve a common use case.

The LLMChain, specifically, is the most common type of chain in LangChain. It consists of a PromptTemplate, a model (which can be either an LLM or a ChatModel), and an optional output parser. The purpose of an LLMChain is to take multiple input variables, format them into a prompt using the PromptTemplate, and then pass that prompt to the specified language model (LLM or ChatModel). The output of the model is then optionally parsed using the provided OutputParser to obtain the final desired format.

LangChain itself is a framework designed for developing applications powered by language models. It aims to go beyond simply using language models through APIs by enabling data-awareness and agentic capabilities. Being data-aware means connecting a languag

The quality of this answer is phenomenal. This is made possible thanks to the idea of augmented our query with external knowledge (source knowledge). There's just one problem — how do we get this information in the first place?

We learned in the previous chapters about Pinecone and vector databases. Well, they can help us here too. But first, we'll need a dataset.

### Importing the Data

In this task, we will be importing our data. We will be using the Hugging Face Datasets library to load our data. Specifically, we will be using the `"jamescalam/llama-2-arxiv-papers"` dataset. This dataset contains a collection of ArXiv papers which will serve as the external knowledge base for our chatbot.

In [2]:
# import fitz

# def load_pdf(file_path):
#     doc = fitz.open(file_path)
#     text = ""
#     for page in doc:
#         text += page.get_text()
#     return text

# # pdf_data = load_pdf('10_Academy_challenge_doc.pdf')

# raw_docs = pdf_data
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

directory_loader = DirectoryLoader("docs", {
            '10_Academy_challenge_doc.pdf': lambda path: PyPDFLoader(path),
        })
raw_docs = directory_loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

docs = text_splitter.split_documents(raw_docs)
print('split docs', docs)

print('creating vector store...')

FileNotFoundError: Directory not found: './docs'

In [38]:
dataset[0]

'1'

#### Dataset Overview

The dataset we are using is sourced from the Llama 2 ArXiv papers. It is a collection of academic papers from ArXiv, a repository of electronic preprints approved for publication after moderation. Each entry in the dataset represents a "chunk" of text from these papers.

Because most **L**arge **L**anguage **M**odels (LLMs) only contain knowledge of the world as it was during training, they cannot answer our questions about Llama 2 — at least not without this data.

### Task 4: Building the Knowledge Base

We now have a dataset that can serve as our chatbot knowledge base. Our next task is to transform that dataset into the knowledge base that our chatbot can use. To do this we must use an embedding model and vector database.

We begin by initializing our connection to Pinecone, this requires a [free API key](https://app.pinecone.io).

In [39]:
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.getenv("PINECONE_API_KEY")
# configure client
pc = Pinecone(api_key=api_key)

Then we initialize the index. We will be using OpenAI's `text-embedding-ada-002` model for creating the embeddings, so we set the `dimension` to `1536`.

In [40]:
from pinecone import PodSpec
import time


index_name = '10-academy-challenge'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    try:
        # if does not exist, create index
        pc.create_index(
            index_name,
            dimension=1536,  # dimensionality of ada 002
            metric='cosine',
            spec=PodSpec(
                environment="gcp-starter"
            )
        )
        # wait for index to be initialized
        while not pc.describe_index(index_name).status['ready']:
            time.sleep(1)
    except Exception as e:
        print(f"Failed to create index: {e}")
        # Handle the error appropriately here

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

Then we connect to the index:

In [41]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

Our index is now ready but it's empty. It is a vector index, so it needs vectors. As mentioned, to create these vector embeddings we will OpenAI's `text-embedding-ada-002` model — we can access it via LangChain like so:

In [35]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

Using this model we can create embeddings like so:

In [36]:
texts = [
    'this is the first chunk of text',
    'then another second chunk of text is here'
]

res = embed_model.embed_documents(texts)
len(res), len(res[0])

(2, 1536)

From this we get two (aligning to our two chunks of text) 1536-dimensional embeddings.

We're now ready to embed and index all our our data! We do this by looping through our dataset and embedding and inserting everything in batches.

In [19]:
from tqdm.auto import tqdm  # for progress bar

data = dataset.to_pandas()  # this makes it easier to iterate over the dataset

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i+batch_size)
    # get batch of data
    batch = data.iloc[i:i_end]
    # generate unique ids for each chunk
    ids = [f"{x['doi']}-{x['chunk-id']}" for i, x in batch.iterrows()]
    # get text to embed
    texts = [x['chunk'] for _, x in batch.iterrows()]
    # embed text
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['chunk'],
         'source': x['source'],
         'title': x['title']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

AttributeError: 'str' object has no attribute 'to_pandas'

We can check that the vector index has been populated using `describe_index_stats` like before:

In [20]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.00013,
 'namespaces': {'': {'vector_count': 13}},
 'total_vector_count': 13}

#### Retrieval Augmented Generation

We've built a fully-fledged knowledge base. Now it's time to connect that knowledge base to our chatbot. To do that we'll be diving back into LangChain and reusing our template prompt from earlier.

To use LangChain here we need to load the LangChain abstraction for a vector index, called a `vectorstore`. We pass in our vector `index` to initialize the object.

In [21]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

c:\Users\Abel\Documents\10 Academy\week-6\Precision-RAG\.venv\lib\site-packages\langchain_community\vectorstores\pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


Using this `vectorstore` we can already query the index and see if we have any relevant information given our question about Llama 2.

In [22]:
query = "What is this challenge week about?"

vectorstore.similarity_search(query, k=3)

[Document(page_content="Performance Indicators: ● Understanding week’s challenge ● Understanding the prompt engineering ● Ability to reuse previous knowledge Tuesday ● RAG components (Rehmet) ● Techniques to improving R (Retrievers) in RAG (Emitnan) Key Performance Indicators: ● Understanding Prompt ranking ● Understanding prompt matching ● Ability to reuse previous knowledge Wednesday ● RAG Evaluation Data Generation (Abel) ● Understanding of prompt matching and ranking (Mahlet) Thursday ● RAG evaluation metrics (Emitnan) ● RAGObs - DevObs of RAG development and production deployment Deliverables NOTE: Document should be a PDF stored in google drive or published blog link. DO NOT SUBMIT A LINK as PDF! If you want to submit pdf document, it should be the content of your report not a link. Interim Submission - Wednesday 8pm UTC ● Link to your code in GitHub ○ Repository where you will be using to complete the tasks in this week's challenge. A minimum requirement is that you have a well 

We return a lot of text here and it's not that clear what we need or what is relevant. Fortunately, our LLM will be able to parse this information much faster than us. All we need is to connect the output from our `vectorstore` to our `chat` chatbot. To do that we can use the same logic as we used earlier.

In [23]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

Using this we produce an augmented prompt:

In [24]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    Performance Indicators: ● Understanding week’s challenge ● Understanding the prompt engineering ● Ability to reuse previous knowledge Tuesday ● RAG components (Rehmet) ● Techniques to improving R (Retrievers) in RAG (Emitnan) Key Performance Indicators: ● Understanding Prompt ranking ● Understanding prompt matching ● Ability to reuse previous knowledge Wednesday ● RAG Evaluation Data Generation (Abel) ● Understanding of prompt matching and ranking (Mahlet) Thursday ● RAG evaluation metrics (Emitnan) ● RAGObs - DevObs of RAG development and production deployment Deliverables NOTE: Document should be a PDF stored in google drive or published blog link. DO NOT SUBMIT A LINK as PDF! If you want to submit pdf document, it should be the content of your report not a link. Interim Submission - Wednesday 8pm UTC ● Link to your code in GitHub ○ Repository where you will be using to complete the tasks in this week's challenge. A minim

There is still a lot of text here, so let's pass it onto our chat model to see how it performs.

In [25]:
# create a new user prompt
prompt = HumanMessage(
    content=augment_prompt(query)
)
# add to messages
messages.append(prompt)

res = chat(messages)

print(res.content)

c:\Users\Abel\Documents\10 Academy\week-6\Precision-RAG\.venv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


This challenge week is about Automatic Prompt Engineering. The tasks for this week include understanding prompt engineering tools and concepts, familiarizing with language models like GPT-4 and GPT-3.5-Turbo, developing a plan for prompt generation and testing, and setting up a development environment for integrating and testing language models.


We can continue with more Llama 2 questions. Let's try _without_ RAG first:

In [27]:
prompt = HumanMessage(
    content="what are this week challenge?"
)

res = chat(messages + [prompt])
print(res.content)

This week's challenge is focused on Automatic Prompt Engineering. The core tasks include understanding prompt engineering tools and concepts, familiarizing with language models like GPT-4 and GPT-3.5-Turbo, developing a plan for prompt generation and testing, and setting up a suitable development environment for integrating and testing language models. The goal is to acquire knowledge and skills in prompt engineering for language models.


The chatbot is able to respond about Llama 2 thanks to it's conversational history stored in `messages`. However, it doesn't know anything about the safety measures themselves as we have not provided it with that information via the RAG pipeline. Let's try again but with RAG.

In [30]:
prompt = HumanMessage(
    content=augment_prompt(
        "what is the concept of task 3?"
    )
)

res = chat(messages + [prompt])
print(res.content)

The concept of Task 3 in the Automatic Prompt Engineering challenge week is to develop a comprehensive plan for prompt generation and testing. This plan should outline the approach for automated prompt generation, test case creation, and prompt evaluation. It involves strategizing and designing a process to generate prompts, create test cases, and evaluate the effectiveness of prompts.


We get a much more informed response that includes several items missing in the previous non-RAG response, such as "red-teaming", "iterative evaluations", and the intention of the researchers to share this research to help "improve their safety, promoting responsible development in the field".